In [1]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K
import numpy as np

In [40]:
#Encoder
inputs = Input(shape=(28, 28, 1))
x = inputs
x = Conv2D(32, (3, 3), strides = 2, padding = "same")(x)
x = LeakyReLU(alpha = 0.2)(x)
x = BatchNormalization(axis = -1)(x)
x = Conv2D(64, (3, 3), strides = 2, padding = "same")(x)
x = LeakyReLU(alpha = 0.2)(x)
x = BatchNormalization(axis = -1)(x)

volumeSize = K.int_shape(x)
x = Flatten()(x)

latent = Dense(16)(x)
encoder = Model(inputs, latent, name = "encoder")

In [81]:
print(encoder.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 7, 7, 64)          256 

In [76]:
#Decoder
latentInputs = Input(shape=(16, ))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)        
x = Conv2DTranspose(f, (3,3), strides=2, padding="same")(x)
x = LeakyReLU(alpha = 0.2)(x)
x = BatchNormalization(axis = -1)(x)
x = Conv2DTranspose(32, (3,3), strides = 2, padding="same")(x)
x = LeakyReLU(alpha = 0.2)(x)
x = BatchNormalization(axis = -1)(x)
x = Conv2DTranspose(1, (3,3), padding="same")(x)
outputs = Activation("sigmoid")(x)           
decoder = Model(latentInputs, outputs, name="decoder")

In [77]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 16)]              0         
_________________________________________________________________
dense_24 (Dense)             (None, 3136)              53312     
_________________________________________________________________
reshape_17 (Reshape)         (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_44 (Conv2DT (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_39 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_transpose_45 (Conv2DT (None, 28, 28, 32)        1846

In [78]:
#Encoder + Decoder = Autoencoder
autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 16)                69392     
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         109377    
Total params: 178,769
Trainable params: 178,385
Non-trainable params: 384
_________________________________________________________________
